## How to use the ifes_apt_tc_data_modeling library

## Load ifes_apt_tc_data_modeling library

In [1]:
import re
import os
import numpy as np
import h5py
import pandas as pd
from jupyterlab_h5web import H5Web
from ifes_apt_tc_data_modeling.utils.utils import create_isotope_vector, \
    isotope_vector_to_dict_keyword, isotope_vector_to_human_readable_name, \
    isotope_to_hash, hash_to_isotope
from ifes_apt_tc_data_modeling.nexus.nx_ion import NxIon
from ifes_apt_tc_data_modeling.utils.molecular_ions import MolecularIonBuilder
from ifes_apt_tc_data_modeling.utils.utils import MAX_NUMBER_OF_ATOMS_PER_ION
from ifes_apt_tc_data_modeling.utils.nist_isotope_data import isotopes
from ase.data import atomic_numbers, atomic_masses, chemical_symbols

## GPM Rouen ENV system

<div class="alert alert-block alert-danger">An ENV file has to be formatted using the UTF8 character encoding for this reader to function!</div>

In [2]:
prefix = f"{os.getcwd()}/data/env/examples_without_provenance"
fnm = ["ErMnO.env"]

In [4]:
print(isinstance(np.uint16, type))

True


In [ ]:
from ifes_apt_tc_data_modeling.env.env_reader import ReadEnvFileFormat

In [ ]:
env = ReadEnvFileFormat(f"{prefix}/{fnm[0]}")
for m_ion in env.env["molecular_ions"]:
    m_ion.report()

In [ ]:
def get_smart_chemical_symbols():
    priority_queue = []
    for symbol in chemical_symbols:
        if len(symbol) == 2:
            priority_queue.append(symbol)
    for symbol in chemical_symbols:
        if symbol != "X" and len(symbol) == 1:
            priority_queue.append(symbol)
    return priority_queue


with open(f"{prefix}/{fnm[0]}", mode="r", encoding="utf-8") as envf:
    txt = envf.read()
    txt = txt.replace("\r\n", "\n")  # windows to unix EOL conversion
    txt = txt.replace(",", ".")  # use decimal dots instead of comma
    txt_stripped = [line for line in txt.split("\n") if line.strip() != ""]  # and line.startswith("#") is False
    for line in txt_stripped:
        print(line)
    # search for ranging definitions "# Definition of"
    rng_s = None
    rng_e = None
    for idx in np.arange(0, len(txt_stripped)):
        if txt_stripped[idx].startswith("# Definition of") is False:
            continue
        else:
            rng_s = idx
            break
    for idx in np.arange(rng_s + 1, len(txt_stripped)):
        if txt_stripped[idx].startswith("# Atom probe definition") is False:
            continue
        else:
            rng_e = idx
            break
    # print(f"{txt_stripped[rng_s]}\n{txt_stripped[rng_e]}")
    for idx in np.arange(rng_s + 1, rng_e):
        tmp = txt_stripped[idx].split()
        # interpret zeroth token into a list of chemical symbols
        print(f"---->{tmp}")
        lst = []
        if tmp[0] == "Hyd":
            lst = []
        elif tmp[0] in get_smart_chemical_symbols():
            lst.append(tmp[0])
        else:            
            tokens = re.split(r'(\d+)', tmp[0])
            for jdx in np.arange(0, len(tokens)):
                kdx = 0
                for sym in get_smart_chemical_symbols():
                    if tokens[jdx][kdx:].startswith(sym) is True:
                        mult = 1
                        if jdx < len(tokens) - 1:
                            if (tokens[jdx][kdx:] == sym) and (tokens[jdx + 1].isdigit() is True):
                                mult = int(tokens[jdx + 1])
                                kdx += len(tokens[jdx + 1])
                        # else:
                        #     print("jdx >")
                        for cnt in np.arange(0, mult):
                            lst.append(sym)
                        kdx += len(sym)
        print(lst)

## FAU/Erlangen pyccapt control/calibration/ranging module

In [ ]:
# eventually change needed point to the location of the files to analyze
prefix = f"{os.getcwd()}/../../../../../paper_paper_paper/joss_nomad_apt/bb_analysis/data"
fnm = ["1748_Nov-14-2023_13-31_Al.h5",
       "1748_Al.h5",
       "1748_Al_range_.h5"]
# df = pd.read_hdf(f"{prefix}/ger_erlangen_pyccapt_format/{fnm[1]}")
# H5Web(f"{prefix}/ger_erlangen_pyccapt_format/{fnm[1]}")
# df

In [ ]:
from ifes_apt_tc_data_modeling.pyccapt.pyccapt_reader import ReadPyccaptControlFileFormat, ReadPyccaptCalibrationFileFormat, ReadPyccaptRangingFileFormat

In [ ]:
# the actual measurement (comparable to LEAP's STR/RRAW/RHIT/HITS)
pyc_m = ReadPyccaptControlFileFormat(f"{prefix}/ger_erlangen_pyccapt_format/{fnm[0]}")

# the calibrations (voltage, bowl, m/q, comparable to LEAP's ROOT and POS/EPOS)
pyc_c = ReadPyccaptCalibrationFileFormat(f"{prefix}/ger_erlangen_pyccapt_format/{fnm[1]}")
xyz = pyc_c.get_reconstructed_positions()
print(xyz.typed_value)
m_q = pyc_c.get_mass_to_charge_state_ratio()
print(m_q.typed_value)

# the ranging definitions (comparable to RNG/RRNG)
pyc_r = ReadPyccaptRangingFileFormat(f"{prefix}/ger_erlangen_pyccapt_format/{fnm[2]}")

## ATO

In [ ]:
from ifes_apt_tc_data_modeling.ato.ato_reader import ReadAtoFileFormat

In [ ]:
prefix = f"{os.getcwd()}/../../../../../paper_paper_paper/joss_nomad_apt/bb_analysis/data"

In [ ]:
directory = f"{prefix}/usa_cameca_ato_format"
fnm = ["Si.epos.v3.ATO",
       "Si.epos.v5.ATO"]
# directory = f"{prefix}/bel_leuven_imec/dataverse_files/correlative-APT-EM"
# fnm = ["finfet/finfet_system.ato",
#        "multilayer/multilayer_system.ato"]
# directory = f"{prefix}/fra_rouen_karam"
# fnm = ["Experimental Analysis of LaB6 ; Negative Pulse ; DC Voltage 6.6 kV ; Amplitudes 2.5 kV and 3 kV.ato",
#        "Experimental Analysis of LaB6 ; Positive Pulse ; Amplitude 15 kV ; DC Voltage 3.2 kV and 5 kV.ato",
#        "Experimental Analysis of LaB6 ; Positive Pulse ; DC Voltage 2.5 kV ; Amplirudes 13.5 kV and 15 kV.ato",
#        "Experimental Analysis of LaB6 ; Positive Pulse ; DC Voltage 7 kV ; Amplitudes 6.5 kV and 7.5 kV.ato"]
# directory = f"{prefix}/usa_richland_kruska/kruska/APT"
# fnm = ["R31_11378-v01.ato",
#        "R31_11381-v02.ato",
#        "R31_11553-v01.ato",
#        "R31_11554-v01.ato",
#        "R31_11556-v01.ato"]
print(directory)
for filename in fnm:
    print(os.path.getsize(f"{directory}/{filename}"))

In [ ]:
for filename in fnm:
    print(f"{directory}/{filename}")
    ato = ReadAtoFileFormat(f"{directory}/{filename}")
    
    xyz = ato.get_reconstructed_positions()
    print(xyz.typed_value)
    m_q = ato.get_mass_to_charge_state_ratio()
    print(m_q.typed_value)

## Matlab FIG

In [ ]:
from ifes_apt_tc_data_modeling.fig.fig_reader import ReadFigTxtFileFormat

In [ ]:
fnm = ["R56_01769.rng.fig.txt"]

In [ ]:
directory = f"{os.getcwd()}/data/fig/examples_without_provenance/ger_erlangen_felfer"
print(directory)

In [ ]:
# for filename in os.listdir(directory):
for filename in fnm:
    if filename.lower().endswith(".fig.txt"):
        print(filename)
        if True is True:  # if filename in []:
            a = ReadFigTxtFileFormat(f"{directory}/{filename}")
            # a.fig["molecular_ions"]

            h5w = h5py.File(f"{directory}/{filename}.nxs", "w")
            h5w.create_group("/entry1")
            i = 1
            for ion in a.fig["molecular_ions"]:
                trg = f"/entry1/ion{i}"
                grp = h5w.create_group(trg)
                grp.attrs["NXclass"] = "NXion"
                dst = h5w.create_dataset(f"{trg}/comment", data=ion.comment.typed_value)
                dst = h5w.create_dataset(f"{trg}/color", data=ion.color.typed_value)
                # dst = h5w.create_dataset(f"{trg}/volume", dtype=np.float32, data=0.)
                # dst.attrs["unit"] = "nm^3"
                dst = h5w.create_dataset(f"{trg}/isotope_vector", dtype=np.uint16,
                                         data=ion.isotope_vector.typed_value, 
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{trg}/nuclid_list", dtype=np.uint16,
                                         data=ion.nuclid_list.typed_value,
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{trg}/charge_state", dtype=np.int8,
                                         data=ion.charge_state.typed_value)
                dst = h5w.create_dataset(f"{trg}/name", data=ion.name.typed_value)
                dst = h5w.create_dataset(trg+ "/mass_to_charge_range", dtype=np.float32,
                                         data=ion.ranges.typed_value)
                dst.attrs["unit"] = "Da"

                subgrpnm = f"{trg}/charge_state_model"
                subgrp = h5w.create_group(subgrpnm)
                subgrp.attrs["NXclass"] = "NXprocess"
                dst = h5w.create_dataset(f"{subgrpnm}/min_abundance", dtype=np.float64,
                                         data=ion.charge_state_model["min_abundance"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_abundance_product", dtype=np.float64,
                                         data=ion.charge_state_model["min_abundance_product"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_half_life", dtype=np.float64,
                                         data=ion.charge_state_model["min_half_life"])
                dst.attrs["unit"] = "s"
                dst = h5w.create_dataset(f"{subgrpnm}/sacrifice_isotopic_uniqueness", dtype=np.uint8,
                                         data=ion.charge_state_model["sacrifice_isotopic_uniqueness"])
                dst = h5w.create_dataset(f"{subgrpnm}/isotope_matrix", dtype=np.uint16,
                                         data=ion.charge_state_model["isotope_matrix"],
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{subgrpnm}/charge_state_vector", dtype=np.int8,
                                         data=ion.charge_state_model["charge_state_vector"])
                dst = h5w.create_dataset(f"{subgrpnm}/mass_vector", dtype=np.float64,
                                         data=ion.charge_state_model["mass_vector"])
                dst.attrs["unit"] = "Da"
                dst = h5w.create_dataset(f"{subgrpnm}/nat_abun_prod_vector", dtype=np.float64,
                                         data=ion.charge_state_model["nat_abun_prod_vector"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_half_life_vector", dtype=np.float64,
                                         data=ion.charge_state_model["min_half_life_vector"])
                dst.attrs["unit"] = "s"

                i += 1
            h5w.close()

In [ ]:
H5Web(f"{directory}/{filename}.nxs")

## RRNG

In [ ]:
from ifes_apt_tc_data_modeling.rrng.rrng_reader import ReadRrngFileFormat

In [ ]:
fnm = ["R31_06365-v02.rrng"]

In [ ]:
directory = f"{os.getcwd()}/data/rrng/examples_without_provenance/ger_duesseldorf_kuehbach"
print(directory)

In [ ]:
# for filename in os.listdir(directory):
for filename in fnm:
    if filename.lower().endswith(".rrng"):
        print(filename)
        if True is True:  # if filename in []:
            a = ReadRrngFileFormat(f"{directory}/{filename}")
            # a.rrng["molecular_ions"]

            h5w = h5py.File(f"{directory}/{filename}.nxs", "w")
            h5w.create_group("/entry1")
            i = 1
            for ion in a.rrng["molecular_ions"]:
                trg = f"/entry1/ion{i}"
                grp = h5w.create_group(trg)
                grp.attrs["NXclass"] = "NXion"
                dst = h5w.create_dataset(f"{trg}/comment", data=ion.comment.typed_value)
                dst = h5w.create_dataset(f"{trg}/color", data=ion.color.typed_value)
                dst = h5w.create_dataset(f"{trg}/volume", dtype=np.float32,
                                         data=ion.volume.typed_value)
                dst.attrs["unit"] = "nm^3"
                dst = h5w.create_dataset(f"{trg}/isotope_vector", dtype=np.uint16,
                                         data=ion.isotope_vector.typed_value, 
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{trg}/nuclid_list", dtype=np.uint16,
                                         data=ion.nuclid_list.typed_value,
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{trg}/charge_state", dtype=np.int8,
                                         data=ion.charge_state.typed_value)
                dst = h5w.create_dataset(f"{trg}/name", data=ion.name.typed_value)
                # print(ion.name.typed_value)
                dst = h5w.create_dataset(trg+ "/mass_to_charge_range", dtype=np.float32,
                                         data=ion.ranges.typed_value)
                dst.attrs["unit"] = "Da"

                subgrpnm = f"{trg}/charge_state_model"
                subgrp = h5w.create_group(subgrpnm)
                subgrp.attrs["NXclass"] = "NXprocess"
                dst = h5w.create_dataset(f"{subgrpnm}/min_abundance", dtype=np.float64,
                                         data=ion.charge_state_model["min_abundance"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_abundance_product", dtype=np.float64,
                                         data=ion.charge_state_model["min_abundance_product"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_half_life", dtype=np.float64,
                                         data=ion.charge_state_model["min_half_life"])
                dst.attrs["unit"] = "s"
                dst = h5w.create_dataset(f"{subgrpnm}/sacrifice_isotopic_uniqueness", dtype=np.uint8,
                                         data=ion.charge_state_model["sacrifice_isotopic_uniqueness"])
                dst = h5w.create_dataset(f"{subgrpnm}/isotope_matrix", dtype=np.uint16,
                                         data=ion.charge_state_model["isotope_matrix"],
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{subgrpnm}/charge_state_vector", dtype=np.int8,
                                         data=ion.charge_state_model["charge_state_vector"])
                dst = h5w.create_dataset(f"{subgrpnm}/mass_vector", dtype=np.float64,
                                         data=ion.charge_state_model["mass_vector"])
                dst.attrs["unit"] = "Da"
                dst = h5w.create_dataset(f"{subgrpnm}/nat_abun_prod_vector", dtype=np.float64,
                                         data=ion.charge_state_model["nat_abun_prod_vector"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_half_life_vector", dtype=np.float64,
                                         data=ion.charge_state_model["min_half_life_vector"])
                dst.attrs["unit"] = "s"
                i += 1
            h5w.close()

In [ ]:
H5Web(f"{directory}/{filename}.nxs")

## RNG

In [ ]:
from ifes_apt_tc_data_modeling.rng.rng_reader import ReadRngFileFormat

In [ ]:
fnm = ["SeHoKim_R5076_44076_v02.rng"]

In [ ]:
directory = f"{os.getcwd()}/data/rng/examples_without_provenance/ger_duesseldorf_kuehbach"
print(directory)

In [ ]:
# for filename in os.listdir(directory):
for filename in fnm:
    if filename.lower().endswith(".rng"):
        print(filename)
        if True is True:  # if filename in []:
            a = ReadRngFileFormat(f"{directory}/{filename}")
            # a.rng["molecular_ions"]
            
            h5w = h5py.File(f"{directory}/{filename}.nxs", "w")
            h5w.create_group("/entry")
            i = 1
            for ion in a.rng["molecular_ions"]:
                trg = f"/entry/ion{i}"
                grp = h5w.create_group(trg)
                grp.attrs["NXclass"] = "NXion"
                dst = h5w.create_dataset(f"{trg}/comment", data=ion.comment.typed_value)
                dst = h5w.create_dataset(f"{trg}/color", data=ion.color.typed_value)
                dst = h5w.create_dataset(f"{trg}/volume", dtype=np.float32,
                                         data=ion.volume.typed_value)
                dst.attrs["unit"] = "nm^3"
                dst = h5w.create_dataset(f"{trg}/isotope_vector", dtype=np.uint16,
                                         data=ion.isotope_vector.typed_value, 
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{trg}/nuclid_list", dtype=np.uint16,
                                         data=ion.nuclid_list.typed_value,
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{trg}/charge_state", dtype=np.int8,
                                         data=ion.charge_state.typed_value)
                dst = h5w.create_dataset(f"{trg}/name", data=ion.name.typed_value)
                dst = h5w.create_dataset(trg+ "/mass_to_charge_range", dtype=np.float32,
                                         data=ion.ranges.typed_value)
                dst.attrs["unit"] = "Da"

                subgrpnm = f"{trg}/charge_state_model"
                subgrp = h5w.create_group(subgrpnm)
                subgrp.attrs["NXclass"] = "NXprocess"
                dst = h5w.create_dataset(f"{subgrpnm}/min_abundance", dtype=np.float64,
                                         data=ion.charge_state_model["min_abundance"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_abundance_product", dtype=np.float64,
                                         data=ion.charge_state_model["min_abundance_product"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_half_life", dtype=np.float64,
                                         data=ion.charge_state_model["min_half_life"])
                dst.attrs["unit"] = "s"
                dst = h5w.create_dataset(f"{subgrpnm}/sacrifice_isotopic_uniqueness", dtype=np.uint8,
                                         data=ion.charge_state_model["sacrifice_isotopic_uniqueness"])
                dst = h5w.create_dataset(f"{subgrpnm}/isotope_matrix", dtype=np.uint16,
                                         data=ion.charge_state_model["isotope_matrix"],
                                         chunks=True, compression="gzip", compression_opts=1)
                dst = h5w.create_dataset(f"{subgrpnm}/charge_state_vector", dtype=np.int8,
                                         data=ion.charge_state_model["charge_state_vector"])
                dst = h5w.create_dataset(f"{subgrpnm}/mass_vector", dtype=np.float64,
                                         data=ion.charge_state_model["mass_vector"])
                dst.attrs["unit"] = "Da"
                dst = h5w.create_dataset(f"{subgrpnm}/nat_abun_prod_vector", dtype=np.float64,
                                         data=ion.charge_state_model["nat_abun_prod_vector"])
                dst = h5w.create_dataset(f"{subgrpnm}/min_half_life_vector", dtype=np.float64,
                                         data=ion.charge_state_model["min_half_life_vector"])
                dst.attrs["unit"] = "s"
                i += 1
            h5w.close()

In [ ]:
H5Web(f"{directory}/{filename}.nxs")

## POS

In [ ]:
from ifes_apt_tc_data_modeling.pos.pos_reader import ReadPosFileFormat

In [ ]:
fnm = ["R31_06365-v02.pos"]
directory = f"{os.getcwd()}/data/pos/examples_without_provenance"
print(directory)

In [ ]:
for filename in fnm:
    pos = ReadPosFileFormat(f"{directory}/{filename}")
    print(pos.filename)
    print(pos.filesize)
    xyz = pos.get_reconstructed_positions()
    print(np.shape(xyz.typed_value))
    mq = pos.get_mass_to_charge_state_ratio()
    print(np.shape(mq.typed_value))

## ePOS

In [ ]:
from ifes_apt_tc_data_modeling.epos.epos_reader import ReadEposFileFormat

In [ ]:
fnm = ["R18_58152-v02.epos"]
directory = f"{os.getcwd()}/data/epos/examples_without_provenance"
print(directory)

In [ ]:
for filename in fnm:
    epos = ReadEposFileFormat(f"{directory}/{filename}")
    print(epos.filename)
    print(epos.filesize)
    xyz = epos.get_reconstructed_positions()
    print(np.shape(xyz.typed_value))
    mq = epos.get_mass_to_charge_state_ratio()
    print(np.shape(mq.typed_value))
    raw_tof = epos.get_raw_time_of_flight()
    print(np.shape(raw_tof.typed_value))
    st_v = epos.get_standing_voltage()
    print(np.shape(st_v.typed_value))
    p_v = epos.get_pulse_voltage()
    print(np.shape(p_v.typed_value))
    det_xy = epos.get_hit_positions()
    print(np.shape(det_xy.typed_value))
    n_p = epos.get_number_of_pulses()
    print(np.shape(n_p.typed_value))
    h_p = epos.get_ions_per_pulse()
    print(np.shape(h_p.typed_value))

## APT

In [ ]:
from ifes_apt_tc_data_modeling.apt.apt6_reader import ReadAptFileFormat

In [ ]:
fnm = ["70_50_50.apt"]
directory = f"{os.getcwd()}/data/apt/examples_without_provenance"
print(directory)

In [ ]:
for filename in fnm:
    apt = ReadAptFileFormat(f"{directory}/{filename}")
    print(apt.get_metadata_table())
    xyz = apt.get_reconstructed_positions()
    print(np.shape(xyz.typed_value))
    mq = apt.get_mass_to_charge_state_ratio()
    print(np.shape(mq.typed_value))

## Low-level stuff testing the library

In [ ]:
# ivec = create_isotope_vector(['K-40', 'Xe-126'])  # , 'U', 'H'])
# ivec = create_isotope_vector(['Tc', 'H', 'H'])  # 'C'])
# ivec = create_isotope_vector(['B', 'B', 'B', 'N', 'N', 'Ti'])  # 'N', 'N'])
# ivec = create_isotope_vector([])
ivec = create_isotope_vector(['O', 'O', 'O', 'O'])
ivec = create_isotope_vector(['Fe', 'Fe', 'O', 'O', 'O', 'O', 'O'])
# ivec = create_isotope_vector(['X'])
print(ivec)
# print(isotope_vector_to_dict_keyword(ivec))
# print(isotope_vector_to_human_readable_name(ivec, +4))

## Take the information available in every range file.

In [ ]:
ion = NxIon(isotope_vector=ivec, charge=0)
# ion.add_range((np.floor(atomic_masses[43]) - 2.)*0.5, (np.floor(atomic_masses[43]) + 4.)*0.5)
# ion.add_range(20.2840, 20.3680)
# ion.add_range(54.9540, 54.9630)
# ion.add_range(32.0050,  32.1370)
# ion.add_range(32.4280,  32.4910)
# ion.add_range(32.9600,  33.0640)
# ion.add_range(33.9720,  34.0380)
# ion.add_range(175.8060, 176.2460)
ion.add_range(95.6540, 96.1780)
# ion.add_range(10.5, 13.5)
# ion.add_range(13.0, 13.3)  # should not be prevented because this is equivalent to ion.add_range(10.0, 13.3)
ion.report()

In [ ]:
mion = MolecularIonBuilder(min_abundance=0.,
                           min_abundance_product=0.,
                           min_half_life=np.inf,
                           sacrifice_uniqueness=True,
                           verbose=True)
mion.combinatorics(
    ion.isotope_vector.typed_value,
    ion.ranges.typed_value[0, 0],
    ion.ranges.typed_value[0, 1])

In [ ]:
p, n = hash_to_isotope(1)
print(f"{p}, {n}")

***
Markus Kühbach 2023/05